In [ ]:
import sys
sys.path.append('../library')
sys.path.append('../library/rtde')
import logging
import rtde.rtde as rtde
import rtde.rtde_config as rtde_config
import time
import math
import subprocess
import socket 
import os
from quaternion import quaternion
from transform_coordinate import transform_coordinate
import keyboard
from IPython.display import clear_output

In [ ]:
#logging.basicConfig(level=logging.INFO)

ROBOT_HOST = '192.168.253.1'
ROBOT_PORT = 30004
config_filename = '../config/pose_pilot_configuration_with_button.xml'

UDP_IP = "127.0.0.1"
UDP_PORT = 5005
sock = socket.socket(socket.AF_INET, # Internet
                     socket.SOCK_DGRAM) # UDP
#sock.setblocking(0)
sock.bind((UDP_IP, UDP_PORT))


keep_running = True
#logging.getLogger().setLevel(logging.INFO)

In [ ]:
conf = rtde_config.ConfigFile(config_filename)
state_names, state_types = conf.get_recipe('state')
setp_names, setp_types = conf.get_recipe('setp')
watchdog_names, watchdog_types = conf.get_recipe('watchdog')
button_names, button_types = conf.get_recipe('button')

In [ ]:
con = rtde.RTDE(ROBOT_HOST, ROBOT_PORT)
con.connect()

# get controller version
con.get_controller_version()

# setup recipes
con.send_output_setup(state_names, state_types)
setp = con.send_input_setup(setp_names, setp_types)
watchdog = con.send_input_setup(watchdog_names, watchdog_types)
button = con.send_input_setup(button_names, button_types)

In [ ]:
def setp_to_list(setp):
    list = []
    for i in range(0,6):
        list.append(setp.__dict__["input_double_register_%i" % i])
    return list

def list_to_setp(setp, list):
    for i in range (0,6):
        setp.__dict__["input_double_register_%i" % i] = list[i]
    return setp

def button_to_list(button):
    list = []
    for i in range(0,2):
        list.append(button.__dict__["input_int_register_%i" % (i+1)])
    return list

def list_to_button(button, list):
    for i in range (0,2):
        button.__dict__["input_int_register_%i" % (i+1)] = list[i]
    return button

In [ ]:
# Initialization
# setp
setp.input_double_register_0 = 0
setp.input_double_register_1 = 0
setp.input_double_register_2 = 0
setp.input_double_register_3 = 0
setp.input_double_register_4 = 0
setp.input_double_register_5 = 0

# button
button.input_int_register_1 = 0
button.input_int_register_2 = 0
  
# The function "rtde_set_watchdog" in the "rtde_control_loop.urp" creates a 1 Hz watchdog
watchdog.input_int_register_0 = 0

# controller to robot gripper transformation
tr_con_rob_rot_vec_aangle = [0, math.pi, 0]
tr_con_rob_rot_vec = quaternion.axis_angle_to_quaternion(tr_con_rob_rot_vec_aangle)
tr_con_rob_cor = transform_coordinate([0, 0, 0], quaternion(1, 0, 0, 0), tr_con_rob_rot_vec)

# Button state
tracking = False
last_index_trigger = False

# Force
contact_force_threshold = 10.
rob_in_contact = False

In [ ]:
#start data synchronization
if not con.send_start():
    sys.exit()

# control loop
while keep_running:
    last_tracking = tracking
    
    # receive the current state
    state = con.receive()
    
    net_torque = (state.actual_TCP_force[3]**2 + state.actual_TCP_force[4]**2 + state.actual_TCP_force[5]**2)**0.5
    sock.sendto((str(state.tcp_force_scalar)+","+str(net_torque)), ("127.0.0.1", 11000))
    data, addr = sock.recvfrom(1024) # buffer size is 1024 bytes
    controller_state = [float(s) for s in data.split(',')]
    

        
    
    # to right hand
    controller_state[2] = -controller_state[2]
    controller_state[5] = -controller_state[5]
    controller_state[6] = -controller_state[6]*math.pi/180.0

    # are we in tracking mode?
    index_trigger = controller_state[11] > 0
    if (not last_index_trigger) & index_trigger:
        tracking =  not tracking
    last_index_trigger = index_trigger
    
    # If connection is broken break
    if state is None:
        break;
    
    # Is robot in contact?
    last_rob_in_contact = rob_in_contact
    rob_in_contact =  state.tcp_force_scalar >= contact_force_threshold
    if rob_in_contact and not last_rob_in_contact:
                    contact_z = state.actual_TCP_pose[2]
                    #print(contact_z)
            
    
    # get new robot set point
    if tracking:
        if not last_tracking:
            # set current controller loc-ori to current rob loc-ori
            # get controller coordinates transformations
            con_base_loc = controller_state[0:3]
            con_base_ori_aangle = [i*controller_state[6] for i in controller_state[3:6]]
            con_base_ori = quaternion.axis_angle_to_quaternion(con_base_ori_aangle)

            tr_con_cor = transform_coordinate(con_base_loc, con_base_ori, con_base_ori) 
            
            # get robot coordinates transformations
            rob_base_loc = state.actual_TCP_pose[0:3]
            rob_base_ori_aangle = state.actual_TCP_pose[3:6]
            rob_base_ori = quaternion.axis_angle_to_quaternion(rob_base_ori_aangle)

            tr_rob_cor = transform_coordinate(rob_base_loc, rob_base_ori, rob_base_ori)
            new_setp = state.actual_TCP_pose
        else:
            # Controller Current Orientation in Controller cordinates
            con_loc = controller_state[0:3]
            con_ori_aangle = [i*controller_state[6] for i in controller_state[3:6]]
            con_ori = quaternion.axis_angle_to_quaternion(con_ori_aangle)

            rel_con_loc, rel_con_ori = tr_con_cor.to_new(con_loc, con_ori)
            rel_rob_loc, rel_rob_ori = tr_con_rob_cor.to_new(rel_con_loc, rel_con_ori)
            rob_loc, rob_ori = tr_rob_cor.to_old(rel_rob_loc, rel_rob_ori)

            # Z correction for contact
            if rob_in_contact and rob_loc[2] <= contact_z:
                rob_loc[2] = contact_z
            
            new_setp = rob_loc + rob_ori.to_axis_angle()
    else:
        new_setp = state.actual_TCP_pose
    
    list_to_setp(setp, new_setp)
    
    # send new setpoint        
    con.send(setp)
    
    # Get button data
    new_button = [0, 0]
    # A close
    if controller_state[7] > 0:
            new_button[0] = 1
    # B open
    if controller_state[8] > 0:
            new_button[1] = 1
    list_to_button(button, new_button)
    
    # Send button data
    con.send(button)
    
    # kick watchdog
    con.send(watchdog)

In [ ]:
con.send_pause()
con.disconnect()
sys.stdout.flush()